# Coeficientes de reflexión a incidencia normal para distintos 'targets'

In [1]:
import numpy as np
from matplotlib import pyplot as plt

## Target semi-infinito con constante dielectrica $\epsilon$

In [2]:
def R_1capa(ep):
    '''
    Coeficiente de reflexion de fresnell (capa seminf)
    obs: no depende de la frecuencia
  
    Entradas:
    -ep: Epsilon (cte dielectrica del medio)

    Salidas:
    -R
    '''
    r=(np.sqrt(ep)-1)/(np.sqrt(ep)+1)
    return r

In [5]:
#LE FALTA PERFECCIONAMIENTO

def S_teo_A(altura, f, S1, S2, ep_teo, n_mv):
    ind = []
    for i in range(len(altura)):
        aux = np.zeros(n_mv,dtype=float)
        for j in range(n_mv):
            ep = ep_teo[j]
        
            #coef de reflexion depende solo de ep
            R = R_1capa(ep)
            S2teo_may = S1[i]*R
            error = np.sum(np.abs(np.abs(S2teo_may)-np.abs(S1)))/np.shape(S1)[1]
            ##error
            aux[j] = error  
        ind.append(np.argmin(aux))

    for i in range(len(altura)):
        ind_i = ind[i]
        if ind_i < 9999:
            print(ind_i)
            # mv_op = MV[ind[i]]
            ep_op = ep_teo[ind[i]]
            S_op = S1*R_1capa(ep_op)
            # error_mae = e_mae[i,ind[i]]
        
            plt.plot(f,np.abs(S_op[i]), label='teo')
            plt.plot(f,np.abs(S2[i]), label='med')
            plt.legend()
            plt.show()
        else:
            print('error')
    
    return ep_op, S_op
    

### target de espesor $d$ y constante dieléctrica $\epsilon$ y conductor ideal debajo del dieléctrico.

In [ ]:
def R_ep_cond(ep,d,f): # antes era Ref2. Solo valido para frecuencias positivas.
    '''
    coef dieléctrico ep de espesor d sobre conductor ideal
    '''
    
    ki = 2*np.pi*f/3e8  
    fi = 2*1j*np.sqrt(ep)*ki*d
    
    a = 1-np.sqrt(ep)+(1+np.sqrt(ep))*np.exp(fi)
    b = 1+np.sqrt(ep)+(1-np.sqrt(ep))*np.exp(fi)
    
    return a/b

## target con dos dieléctricos: $\epsilon_1$ con espesor $d$ y luego $\epsilon_2$ semi-infinito

In [ ]:
def R_ep1_ep2(ep1,ep2,d,f):
    '''
    dos dieléctricos: ep1 de espesor d y ep2 semi-infinito.
    
    config: aire-ep1-ep2
    '''
    
    landa = 3e8/f
    r01 = (ep1-1)/(ep1+1)
    r12 = (ep2-ep1)/(ep2+ep1)
    fase = 2*np.sqrt(ep1)*2*np.pi*d/landa
    
    r = (r01+r12*np.exp(1j*fase))/(1+r01*r12*np.exp(1j*fase))
    
    return r
    

### target de espesor $d_1 + d_2 = d$, donde la constante dieléctrica varía en dos tramos: $\epsilon_1$ arriba, $\epsilon_2$ debajo, luego el conductor ideal. 

In [ ]:
def R_2capas(ep1,ep2,d1,d2,f):
    '''
    dos dieléctricos: ep1 con espesor d1, ep2 con espesor d2;
    ep1 sobre ep2 y ep2 sobre dieléctrico.
    
    config: aire-ep1-ep2-conductor.
    '''
    
    k0 = 2*np.pi*f/3e8
    k1 = np.sqrt(ep1)*k0
    k2 = np.sqrt(ep2)*k0
    
    z1 = np.exp(-2*1j*k1*d1)
    z2 = np.exp(-2*1j*k2*d2)

    r01 = (np.sqrt(ep1)-1)/(np.sqrt(ep1)+1) # interface ep0-ep1; ep0 = 1
    r12 = (np.sqrt(ep2)-np.sqrt(ep1))/(np.sqrt(ep2)+np.sqrt(ep1))  # interface ep1-ep2, dentro del dieléctrico
    r2cond = 1 # interface ep2-conductor ideal

    a = r01 + r12/z1 + r01*r12*r2cond/z2 + r2cond/(z1*z2)
    b = 1 + r01*r12/z1 + r12*r2cond/z2 + r01*r2cond/(z1*z2)

    R = a/b

    return R

### Coeficiente de fresnell para 3 capas de dieléctrico

In [6]:
def R_3capas(ep1, ep2, ep3, d1, d2, f):
    '''
  Coeficiente de reflexion de fresnell 2 interfases
    Entradas:
    -ep1: ϵ de la 1ra capa
    -ep2: ϵ de la 2da capa
    -ep: ϵ de la 3ra capa
    -d1: ancho de la 1er capa
    -d2: ancho de la 2da capa
    -f: frecuencia incidente

    Salidas:
    -R
  '''
    phi1 = (2*np.pi*f*np.sqrt(ep1)*d1)/3e8
    phi2 = (2*np.pi*f*np.sqrt(ep2)*d2)/3e8
    R0 = (1-np.sqrt(ep1))/(1+np.sqrt(ep1))
    R1 = (np.sqrt(ep1)-np.sqrt(ep2))/(np.sqrt(ep1)+np.sqrt(ep2))
    R2 = (np.sqrt(ep2)-np.sqrt(ep3))/(np.sqrt(ep2)+np.sqrt(ep3))
    
    RR1=(R1+R2*np.exp(-2*1j*phi2))/(1+R1*R2*np.exp(-2*1j*phi2))
    
    a = R0+RR1*np.exp(-2*1j*phi1)
    b = 1+R0*RR1*np.exp(-2*1j*phi1)
    
    R = a/b
    return R